In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd


In [37]:
bike_data = pd.read_csv('data/daily-bike-share.csv')
bike_data.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,rentals
0,1,1/1/2011,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331
1,2,1/2/2011,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131
2,3,1/3/2011,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120
3,4,1/4/2011,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108
4,5,1/5/2011,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82


## Select inputs and target


In [72]:
selected_features = ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']
target = 'rentals'
data = bike_data[selected_features].values
labels = bike_data[target]



## Normalize the feature

In [63]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(data)

## Split the data into training and testing sets


In [64]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)

## Reshape the input for RNN

In [65]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

## Build the RNN model

In [66]:
model = Sequential()
model.add(SimpleRNN(units=50, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
model.add(Dense(units=1))

C:\Users\Computer\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [67]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

# Print the model summary
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_5 (SimpleRNN)             │ (None, 50)                  │           3,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,151 (12.31 KB)

 Trainable params: 3,151 (12.31 KB)

 Non-trainable params: 0 (0.00 B)

## Train the Model

In [70]:
# Train the model
history = model.fit(X_train, y_train, epochs=50,validation_split=0.2, batch_size=32,verbose=2)



Epoch 1/50
15/15 - 0s - 16ms/step - loss: 507145.0312 - val_loss: 404108.0312
Epoch 2/50
15/15 - 0s - 8ms/step - loss: 504708.5000 - val_loss: 402207.0000
Epoch 3/50
15/15 - 0s - 8ms/step - loss: 502481.4062 - val_loss: 400316.5938
Epoch 4/50
15/15 - 0s - 8ms/step - loss: 500258.8438 - val_loss: 398527.4375
Epoch 5/50
15/15 - 0s - 8ms/step - loss: 498099.6562 - val_loss: 396733.0938
Epoch 6/50
15/15 - 0s - 9ms/step - loss: 495973.6562 - val_loss: 394970.4062
Epoch 7/50
15/15 - 0s - 9ms/step - loss: 493913.8438 - val_loss: 393084.3438
Epoch 8/50
15/15 - 0s - 8ms/step - loss: 491822.3750 - val_loss: 391328.0938
Epoch 9/50
15/15 - 0s - 8ms/step - loss: 489713.3125 - val_loss: 389705.7500
Epoch 10/50
15/15 - 0s - 9ms/step - loss: 487709.4375 - val_loss: 388109.1562
Epoch 11/50
15/15 - 0s - 9ms/step - loss: 485752.8438 - val_loss: 386566.3438
Epoch 12/50
15/15 - 0s - 9ms/step - loss: 483837.0625 - val_loss: 384941.1250
Epoch 13/50
15/15 - 0s - 8ms/step - loss: 481985.7188 - val_loss: 383308

## Make Predictions

In [74]:
# Make predictions
predictions = model.predict(X_test)

# Print predictions
print("Predictions:", predictions[:5])
print("True Values:", test_labels[:5])

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Predictions: [[1042.3796 ]
 [ 449.88568]
 [ 843.8773 ]
 [ 746.41144]
 [1030.6904 ]]
True Values: 584    1278
585    1263
586    1196
587    1065
588    2247
Name: rentals, dtype: int64
